Import Libraries

In [1]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import numpy as np


Define the Dataset Class

In [2]:
class MaskedUnmaskedDataset(Dataset):
    def __init__(self, masked_dir, unmasked_dir, transform=None):
        self.masked_dir = masked_dir
        self.unmasked_dir = unmasked_dir
        self.transform = transform
        self.masked_images = os.listdir(masked_dir)
        self.unmasked_images = os.listdir(unmasked_dir)
        self.images = self.masked_images + self.unmasked_images
        self.labels = [1] * len(self.masked_images) + [0] * len(self.unmasked_images)  # 1 for masked, 0 for unmasked

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if idx < len(self.masked_images):
            img_path = os.path.join(self.masked_dir, self.masked_images[idx])
        else:
            img_path = os.path.join(self.unmasked_dir, self.unmasked_images[idx - len(self.masked_images)])

        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


Image Transformations

In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the images to 128x128
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])


Load Dataset

In [4]:
# masked_dir = r'C:\Users\USER\OneDrive\Desktop\Masked Face Detection using Periocular Region\Masked_LFW_Dataset'  # Updated path
# unmasked_dir = r'C:\Users\USER\OneDrive\Desktop\Masked Face Detection using Periocular Region\LFW_without_Mask'  # Updated path
masked_dir = os.getcwd()+"\\Masked_LFW_Dataset\\Masked_LFW_Dataset"
unmasked_dir = os.getcwd()+"\\LFW_without_Mask\\LFW_without_Mask"
dataset = MaskedUnmaskedDataset(masked_dir, unmasked_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Continue with your training loop...


Define CNN Model

In [5]:
model = models.resnet50(pretrained=True)

# Modify the final layer to match the number of classes in  dataset (Masked and Unmasked - 2 classes)
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=2)


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


 Training Loop
Here is the training loop that will run for a set number of epochs.

In [7]:
epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(data_loader)}")


Epoch [1/10], Loss: 0.020064705423711818
Epoch [2/10], Loss: 0.00560422752966214
Epoch [3/10], Loss: 0.00010291419536854867
Epoch [4/10], Loss: 2.9157192567825322e-05
Epoch [5/10], Loss: 1.1651824673370624e-05
Epoch [6/10], Loss: 1.0145495652026047e-05
Epoch [7/10], Loss: 1.4162202586784722e-05
Epoch [8/10], Loss: 1.0149122007455597e-05
Epoch [9/10], Loss: 6.7722102936957445e-06
Epoch [10/10], Loss: 3.6952404139518234e-06


Understanding the Loss Values
Loss Function:

The loss function measures how well the model's predictions match the true labels. Lower loss values indicate better performance.
In your case, you're using Cross-Entropy Loss, which is appropriate for multi-class classification problems (like identifying masked vs. unmasked faces).
Epoch Results:

Epoch 1: Loss starts at approximately 0.0233. This is a relatively low value, indicating that the model is already making good predictions early on.
Subsequent Epochs: The loss decreases rapidly over the first few epochs, indicating that the model is learning effectively. By Epoch 10, the loss is around 5.61e-06, which is extremely low.
Convergence: The rapid drop in loss suggests that the model has quickly learned the distinctions in the dataset. After the initial epochs, the decrease in loss is very gradual, indicating that the model may be nearing convergence (i.e., it has learned most of what it can from the training data).
Interpretation of Results
Overfitting Concerns: Given the very low loss values, it's possible that the model might be overfitting the training data, especially if your dataset is small. Overfitting occurs when the model learns to memorize the training data instead of generalizing to unseen data. This can be a concern if you see a significant drop in performance when testing on validation or test datasets.



Model Evaluation
After training, i evaluated the model using the evaluation code you provided. This part calculates the model's accuracy on the dataset, giving insight into its performance:



In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in data_loader:  # You can use a separate test loader here
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the model: {100 * correct / total}%")


Accuracy of the model: 100.0%


Validate the Model:

In [9]:
# Assuming you have split your dataset
from sklearn.model_selection import train_test_split

train_dataset, val_dataset = train_test_split(dataset, test_size=0.2)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Splitting the Dataset into Training and Validation Sets:
We'll use train_test_split from sklearn.model_selection to split the dataset into training and validation sets.



In [10]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Get the indices for train and validation sets
train_indices, val_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)

# Use Subset to create new Datasets for training and validation
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

# Create DataLoaders for both datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Modify Training Loop to Include Validation:
Now, we will adjust the training loop to track both training loss and validation loss after each epoch. This will help us monitor if the model is overfitting.

In [11]:
print = ...
del print


In [12]:
# Make sure 'print' is not overridden
try:
    del print
except NameError:
    pass

import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import numpy as np

# Dataset Class
class MaskedUnmaskedDataset(Dataset):
    def __init__(self, masked_dir, unmasked_dir, transform=None):
        self.masked_dir = masked_dir
        self.unmasked_dir = unmasked_dir
        self.transform = transform
        self.masked_images = os.listdir(masked_dir)
        self.unmasked_images = os.listdir(unmasked_dir)
        self.images = self.masked_images + self.unmasked_images
        self.labels = [1] * len(self.masked_images) + [0] * len(self.unmasked_images)  # 1 for masked, 0 for unmasked

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if idx < len(self.masked_images):
            img_path = os.path.join(self.masked_dir, self.masked_images[idx])
        else:
            img_path = os.path.join(self.unmasked_dir, self.unmasked_images[idx - len(self.masked_images)])

        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Image Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the images to 128x128
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load Dataset
masked_dir = os.getcwd() + "\\Masked_LFW_Dataset\\Masked_LFW_Dataset"
unmasked_dir = os.getcwd() + "\\LFW_without_Mask\\LFW_without_Mask"
dataset = MaskedUnmaskedDataset(masked_dir, unmasked_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define CNN Model
model = models.resnet50(pretrained=True)

# Modify the final layer to match the number of classes (Masked and Unmasked - 2 classes)
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(data_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}")

# Model Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in data_loader:  # You can use a separate test loader here
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy of the model: {accuracy:.2f}%")


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Train Loss: 0.0270
Epoch [2/10], Train Loss: 0.0200
Epoch [3/10], Train Loss: 0.0001
Epoch [4/10], Train Loss: 0.0000
Epoch [5/10], Train Loss: 0.0000
Epoch [6/10], Train Loss: 0.0000
Epoch [7/10], Train Loss: 0.0000
Epoch [8/10], Train Loss: 0.0215
Epoch [9/10], Train Loss: 0.0153
Epoch [10/10], Train Loss: 0.0024
Accuracy of the model: 100.00%


In [13]:
# Look for something like this:
print = "some string"  # This would override the built-in print()

# Rename the variable to avoid conflict:
log_message = "some string"  # Corrected variable name


Test on Unseen Data (Test Set Evaluation)

In [14]:
# Split dataset into training and testing sets (80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoader for test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Recreate DataLoader for train dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


 Cross-Validation (K-fold Cross Validation)

Add Regularization (Dropout or L2 Regularization)

In [18]:
# Add Dropout in the model
model = models.resnet50(pretrained=True)

# Add dropout before the fully connected layer
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.5),  # Dropout with probability of 50%
    torch.nn.Linear(in_features=model.fc.in_features, out_features=2)  # Final layer for 2 classes
)

# Continue training as before
#L2 Regularization (Weight Decay):
# can add L2 regularization by passing the weight_decay parameter to  optimizer.

#python

Data Augmentation (Improve Generalization)

In [19]:
# Update transform to include data augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(15),  # Randomly rotate the image by 15 degrees
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Simplify Model (Try a Simpler Model)
ResNet50 is quite powerful.  might try a simpler architecture like a smaller CNN (Convolutional Neural Network) model.

In [20]:
class SimpleCNN(torch.nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(32 * 32 * 32, 128)  # 128x128 input -> 32x32 after pooling twice
        self.fc2 = torch.nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)  # Flatten the tensor
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate and use this model instead of ResNet50
model = SimpleCNN()


Learning Rate and Epochs (Early Stopping)

In [21]:
best_val_loss = float('inf')
patience = 3  # Stop after 3 epochs with no improvement
trigger_times = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Early Stopping Logic
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


TypeError: 'str' object is not callable

### Predictions and Saving to CSV

In [ ]:
import pandas as pd

# Put the model in evaluation mode
model.eval()

# Initialize an empty list to store the predictions
predictions = []
image_ids = []

# Loop over the test data and make predictions
with torch.no_grad():
    for images, ids in test_loader:  # Assuming 'ids' are your image IDs or identifiers
        images = images.to(device)
        outputs = model(images)
        
        # Get the predicted class (index of the max log-probability)
        _, predicted = torch.max(outputs.data, 1)
        
        # Convert predictions and IDs to CPU, if needed
        predicted = predicted.cpu().numpy()
        ids = ids.cpu().numpy()

        # Append predictions and ids to the list
        predictions.extend(predicted)
        image_ids.extend(ids)

# Create a DataFrame for the predictions
df_predictions = pd.DataFrame({
    'Image_ID': image_ids,  # Replace 'Image_ID' with the actual identifier
    'Predicted_Class': predictions
})

# Save the predictions as a CSV file
df_predictions.to_csv('model_predictions.csv', index=False)

print("Predictions saved to 'model_predictions.csv'.")


In [ ]:
# Save model parameters (state dict)
torch.save(model.state_dict(), 'model.pth')
print("Model saved as 'model.pth'.")


#### Showcase Full Advanced Deep Learning Proficiency

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

# Use your dataset directories
masked_dir = os.getcwd()+"\\Masked_LFW_Dataset\\Masked_LFW_Dataset"
unmasked_dir = os.getcwd()+"\\LFW_without_Mask\\LFW_without_Mask"

# 1. Define transformations for both training and testing sets
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 2. Load datasets using ImageFolder (assuming a folder structure like masked_dir/class1/, unmasked_dir/class2/)
train_dataset = torchvision.datasets.ImageFolder(root=masked_dir, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=unmasked_dir, transform=transform)

# 3. Create DataLoaders for training and testing
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# 4. Load a Pretrained Model (ResNet50) and modify the final layer for binary classification (masked/unmasked)
model = models.resnet50(pretrained=True)

# Freeze the layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)  # For binary classification
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 5. Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, weight_decay=1e-5)

# 6. Training loop with validation
train_losses, val_losses = [], []

for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation step
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(test_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# 7. Plot the training and validation loss curves
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 8. Making Predictions and Saving to CSV
model.eval()

predictions = []
image_ids = []

with torch.no_grad():
    for images, ids in test_loader:  # Assuming 'ids' are your image identifiers
        images = images.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu().numpy()
        ids = ids.cpu().numpy()  # Assuming image IDs are passed

        predictions.extend(predicted)
        image_ids.extend(ids)

# Save predictions to CSV
df_predictions = pd.DataFrame({
    'Image_ID': image_ids,  # Replace 'Image_ID' with actual identifier if needed
    'Predicted_Class': predictions
})

df_predictions.to_csv('model_predictions.csv', index=False)
print("Predictions saved to 'model_predictions.csv'.")
